In [1]:
import copy

import numpy as np
from matplotlib import pyplot as plt
#Import the needed opentps.core packages
import time
import os
import logging
import math
import numpy as np
from matplotlib import pyplot as plt
import sys
from opentps.core.io.dicomIO import writeRTPlan
from opentps.core.processing.planOptimization.tools import evaluateClinical
from opentps.core.data.images import CTImage
from opentps.core.data.images import ROIMask
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.data import Patient
from opentps.core.data.plan import FidObjective
from opentps.core.io.scannerReader import readScanner
from opentps.core.io.serializedObjectIO import saveRTPlan, loadRTPlan
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D, resampleImage3D
from opentps.core.processing.C_libraries.libRayTracing_wrapper import WET_raytracing
from opentps.core.processing.planOptimization.planOptimization import IMPTPlanOptimizer
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.io import mcsquareIO
from opentps.core.io.mcsquareIO import _dicomIsocenterToMCsquare
from opentps.core.io.scannerReader import readScanner
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.io.dataLoader import readData
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import FidObjective
from opentps.core.io.serializedObjectIO import saveBeamlets, saveRTPlan, loadBeamlets, loadRTPlan

02/09/2025 11:35:32 AM - root - INFO - Loading logging configuration: E:\opentps-v1.1.1\opentps_core\opentps\core\config\logger\logging_config.json
02/09/2025 11:35:32 AM - opentps.core._loggingConfig - INFO - Log level set: INFO


d:\Anaconda\envs\OpenTPS_v1\lib\site-packages\cupyx\jit\_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [2]:
mc2 = MCsquareDoseCalculator()
print(mc2._workDir)

C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\Outputs


In [3]:
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL(DoseCalculationConfig().bdlFile)
ctImagePath = "E:\OpenTPS\opentps-master\opentps_core\opentps\core\examples\planOptimization\patient5"
data = readData(ctImagePath)
#print(data)
# #proton:RD RT RS CT
# Photon:RD RD RD RD RD RS CT
rt_struct = data[-2]
ct = data[-1]
rt_struct.print_ROINames()
print("CT spacing=",ct.spacing)
print("CT grid size",ct.gridSize)
target_name = "GTV"
target = rt_struct.getContourByName(target_name).getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
CHIASM = rt_struct.getContourByName("CHIASM.BF").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
NRV_R = rt_struct.getContourByName("RT OPT NRV.BF").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
NRV_L = rt_struct.getContourByName("LT OPT NRV.BF").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
CORD = rt_struct.getContourByName("SPINALCORD").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing) 
Brainstem = rt_struct.getContourByName("brain stem").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing) 
LT_PAROTID = rt_struct.getContourByName("LT PAROTID.BF").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing) 
RT_PAROTID = rt_struct.getContourByName("RT PAROTID.BF").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing) 
ring=target.createMaskRings(3,10)
ring10=ring[0]
ring20=ring[1]
ring30=ring[2]
roi=target

#OAR_Cord = rt_struct.getContourByName("Cord").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
#
COM_coord = target.centerOfMass
COM_index = target.getVoxelIndexFromPosition(COM_coord)
Z_COORD = COM_index[2]
iso_pos=_dicomIsocenterToMCsquare(roi.centerOfMass, ct.origin, ct.spacing, ct.gridSize)
iso_roi=roi.centerOfMass

#
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.ctCalibration = ctCalibration


RT Struct UID: 1.2.826.0.1.3680043.8.274.1.1.8323328.61652.1538400815.323822
  [0]  skin
  [1]  GTV
  [2]  CTV+2mm
  [3]  CTV
  [4]  ODONTOID
  [5]  LT. MASTOID
  [6]  RT. MASTOID
  [7]  brain stem
  [8]  SPINALCORD
  [9]  CHIASM.BF
  [10]  RT OPT NRV.BF
  [11]  LT OPT NRV.BF
  [12]  RT COCHLEA.BF
  [13]  LT COCHLEA.BF
  [14]  LT PAROTID.BF
  [15]  RT PAROTID.BF
  [16]  PITUIT.BF
CT spacing= [0.684 0.684 2.5  ]
CT grid size [512 512  80]
02/09/2025 11:25:47 AM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.
02/09/2025 11:25:48 AM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.
02/09/2025 11:25:49 AM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.


In [5]:
gantryAngles = [90,270]
couchAngles = [0,0]
beamNames = []
for i in range(len(gantryAngles)):
    beamNames.append("Beam%s" % i)
planDesign = PlanDesign()
planDesign.ct = ct
planDesign.gantryAngles = gantryAngles
planDesign.targetMask = roi
planDesign.beamNames = beamNames
planDesign.couchAngles = couchAngles
planDesign.calibration = ctCalibration
planDesign.spotSpacing = 5.0
planDesign.layerSpacing = 5.0
planDesign.targetMargin = 3.0
plan = planDesign.buildPlan()  # Spot placement
plan.PlanName = "NewPlan"

02/09/2025 11:26:19 AM - opentps.core.data.plan._planDesign - INFO - Building plan ...
02/09/2025 11:26:19 AM - opentps.core.processing.planOptimization.planInitializer - INFO - Target is dilated using a margin of 3.0 mm. This process might take some time.
02/09/2025 11:26:19 AM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.
02/09/2025 11:26:32 AM - opentps.core.data.plan._planDesign - INFO - New plan created in 13.130740404129028 sec
02/09/2025 11:26:32 AM - opentps.core.data.plan._planDesign - INFO - Number of spots: 1345


In [ ]:
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.nbPrimaries = 5e4
mc2.ctCalibration = ctCalibration
beamlets,beamletsLET = mc2.computeBeamletsAndLET(ct, plan)
plan.planDesign.beamlets = beamlets

02/09/2025 11:27:24 AM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Beamlet calculation
02/09/2025 11:27:28 AM - opentps.core.io.mhdIO - INFO - Write MHD file: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\CT.mhd
02/09/2025 11:27:28 AM - opentps.core.io.mcsquareIO - INFO - Write plan: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\PlanPencil.txt
02/09/2025 11:27:30 AM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


In [ ]:
#########################################                  Plan Opti  TG119 prostate                                  ###############################################################
plan.planDesign.objectives = ObjectivesList()
plan.planDesign.objectives.setTarget(roi.name,roi, 55.0)
plan.planDesign.objectives.fidObjList = []
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.DVHMAX, 58.0, 100.0,volume=5)
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.DMAX, 59.0, 50.0,robust=True)
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.DVHMIN, 55.0, 100.0,volume=95)
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.DMIN, 54.0, 100.0,robust=True)
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.LETMEAN,
    limitValue=6,
    weight=10.0,
    robust=False  # LET 仅非鲁棒生效
)
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.LETMIN,4,10)
solver = IMPTPlanOptimizer(method='Scipy_L-BFGS-B', plan=plan, maxiter=300,useLET=True)

solver.thresholdSpotRemoval=0
plan.planDesign.beamletsLET=beamletsLET
# Optimize treatment plan
doseImage, ps  = solver.optimize()
doseImage.patient = plan.patient
#########################################                  Dicom Plan  output         ######################################################################
patient = Patient()
patient.name = 'Patient'
plan.patient = patient
output_path = os.getcwd()
#writeRTPlan(plan,output_path,"Arcplan_ELSA_90GA_Lung_Opti.dcm")
#saveRTPlan(plan,output_path+"/Arcplan_CSS_best_90GA_Lung_ROB_Opti.tps")


01/09/2025 04:35:23 PM - opentps.core.processing.planOptimization.planOptimization - INFO - Prepare optimization ...
01/09/2025 04:35:32 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - Iteration 1 of Scipy-L-BFGS-B
01/09/2025 04:35:32 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - objective = 3.280291e+04  
01/09/2025 04:35:34 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - Iteration 2 of Scipy-L-BFGS-B
01/09/2025 04:35:34 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - objective = 2.561937e+04  
01/09/2025 04:35:36 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - Iteration 3 of Scipy-L-BFGS-B
01/09/2025 04:35:36 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - objective = 2.471338e+04  
01/09/2025 04:35:38 PM - opentps.core.processing.planOptimization.solvers.scipyOpt - INFO - Iteration 4 of Scipy-L-BFGS-B
01/09/2025 04:35:38 PM - opentps.core.